# 1. Information about the submission

## 1.1 Name and number of the assignment 

2.2 Text Detoxification

## 1.2 Student name

Petr Sokerin

## 1.3 Codalab user ID

petr_sokerin

## 1.4 Additional comments

***Enter here** any additional comments which you would like to communicate to a TA who is going to grade this work not related to the content of your submission.*

# 2. Technical Report

*Use Section 2 to describe results of your experiments as you would do writing a paper about your results. DO NOT insert code in this part. Only insert plots and tables summarizing results as needed. Use formulas if needed do described your methodology. The code is provided in Section 3.*

## 2.1 Methodology 

In this task I tried to provide tetoxication of russian texts. There are 2 main strategies in this tasks. Fists strategy is dropping toxic words or changing them on intoxics synonyms. The second one is retrieve: changy the style of the entire texts. It can be achieved by modern methods like neural networks. 

It this reseach I implement both strategies with different methods:

1. Deleting or changing toxic words with embeddings Roberta. In this 2 methods I firstly build a vocabulary of toxic words with level of 'toxity' for every word in dataset. 

As tokenizer I used pretrained for Roberta tokenizer to get embeddings of words. As a lematization tool I used Mystemmer.lemmatization from module pymystem3.After preparing texts I provide the prosedure of finding toxic words in sentences: with chosen threshold (0.5) I mark all words as toxic or not. Level of "toxity" of words is not less than zero and no more than one. It acn be interpretated as "probability". The level of threshholds was chosen with different experiments: ther level of thresholds return worse results. 

After that I make one of 2 possible steps:

- changing toxic word with intoxic (level of toxity less then threshold) synonym. To find synonyms I use model KDTree from sklearn.

- dropping toxic words. All toxic words just was deleted from sentences. It can be done because ordinary toxic words use as adjectives or words that not playA key role in sentences. 

2. Using language model to change structure of sentence reducing toxicity with pretrained model. This idea can be good because different language models like GPT transformats texts to less toxic level. Toxic sentences rarely used as in train datasets so language models can quite easly reduce toxity of texts. 

Model, that was used in this try, is detoxGPT from Skoltech. It based on architecture of ruGPT from Sberbank, but finetuned for solving task of detoxication of texts. 

3. Used Encoder-deconer teural network for translation from informal style to formal style. 

Encoder-decoder architechure was used as one of the most appropriate for machine translation that has a lot in common with solving detoxication tasks. I implemented 2 version of the model: without attention mechanism and with it. 

![image](https://www.microsoft.com/en-us/research/uploads/prod/2019/06/Pre-Training-Method-MASS_Site_06_2019_1400x788-1280x720.png)

This neural network architecture include 2 base models: encoder model and decoder model. The role of encoder model is to transform the information to a bector implementation. The idea of decoder model is use outputs of encoder and recover data in the different form. This moodel cam be quiet useful because in the machiene translation scientists tried to use special language interlingua for the no-neural translation. This model do the similar: it get information, translate it to a mediating state end transform it back. 

It can be realize with or without attention mechanism. The role of attention is in fitting and emphasizing more important detailes it solving task.

For solving this task I used WordPunctTokenizer as a tockenizer instrument, split data for train and test (test size was 30%);

For training neural network I used mask -1 as padding and special tocken for uknown words.

Architecture of endoder-decoder without attention is:

Model(
  (emb_inp): Embedding(15156, 64)
  (emb_out): Embedding(14992, 64)
  (enc0): GRU(64, 128, batch_first=True)
  (dec0): GRU(64, 128, batch_first=True)
  (dec_dense): Linear(in_features=128, out_features=14992, bias=True)
  (logits): LogSoftmax(dim=2)
)

As a loss I implemented cross-entropy loss from pytorch with modifitacions for mask. As an Optimizer I used Adam optimizer. Batch size was equal to 32, learning rate 0.003, number of epoches 5000 (was chosen in different experiments). 

I also implement model with attention mechanism:

Architecture of endoder-decoder with attention is:

AttentiveModelLayer(
  (emb_inp): Embedding(15156, 64)
  (emb_out): Embedding(14992, 64)
  (enc0): GRU(64, 128, batch_first=True)
  (dec0): GRU(64, 128, batch_first=True)
  (dec_dense): Linear(in_features=256, out_features=14992, bias=True)
  (logits): LogSoftmax(dim=2)
  (l_attention): AttentionLayer(
    (activ): Tanh()
    (linear_enc): Linear(in_features=128, out_features=128, bias=True)
    (linear_dec): Linear(in_features=128, out_features=128, bias=True)
    (linear_out): Linear(in_features=128, out_features=1, bias=True)
    (softmax): Softmax(dim=1)
  )
)

Parametrs for a model with attention mechanism is similar to the case without attention, but number of epoch is equal to 7000 (if it was more model start to overfit). 


4. T5-model.

Besides I also tried not only to use not-pretrained model, but pretrained models too. As pretrained models I used T5 model for russian texts and fine tune it with our  train dataset. The architecture of the model is on the picture below:


![image](https://miro.medium.com/max/1400/1*iJcUH1F0TmCQE5p2wQt9og.png)

for fine-tuning I used 4 epochs (was chosen in experiments), batch size was chosen equal to 16, window size was equal 100, learning rate was chosen 0.0001.  

## 2.2 Discussion of results

Method | ChrF1 | Style Transfer Accuracy | Join
--- | --- | --- | ---
Roberta: synonyms | 0.0536 | 0.927 | 0.0497
**Roberta: dropping**  | **0.0605** | **0.932** | **0.0564**
detoxGPT | 0.0101 | 0.878 | 0.0088
my Enc-Dec | 0.0184 | 0.897 | 0.0165
my Enc-Dec-Attent | 0.0342 | 0.996 | 0.0341
T5 fine-tuned | 0.0189 | 0.943 | 0.0178

The results of research not as expected as can be. The best results implement model of dropping toxic words Roberta: dropping. It perform the best in ChrF1 and not so bad in Style Transfer Accuracy. This model is quet good because other model are not so good as can be:

*Roberta: synonyms*. For this model there are one big problemm: it provides best synonym in lemmatized form. Actually it can be solved with saving form of words before lammatization and transforming synonym back. Besides, intoxic synonym can be not appropriated for this case in the sentence.

*detoxGPT*. This model has a big potential, but maybe for better results it can be fine tune with dataset more precisely.

*T5 fine-tuned*. The result of this model is not so good because the corpus for fine tuning is not so big and we need to use augmentation for our dataset or provide the bigger one. We can also try to change the architecture for fine tuning.

*my Enc-Dec-(Attent)* In this 2 models attention mechanism provide nice grove in the quality, that is not surprising. The results is not so good because we have small train dataset. Besides, we can experiment with different architecture of models and tuning hiperparametrs more precisely, but without bigger dataset it can be the lack of time. Model with attention provide very good style transfer accuracy but it's not so good in ChrF1. Marking of toxic words can help in this case. 


# 3. Code

*Enter here all code used to produce your results submitted to Codalab. Add some comments and subsections to navigate though your solution.*

*In this part you are expected to develop yourself a solution of the task and provide a reproducible code:*
- *Using Python 3;*
- *Contains code for installation of all dependencies;*
- *Contains code for downloading of all the datasets used*;
- *Contains the code for reproducing your results (in other words, if a tester downloads your notebook she should be able to run cell-by-cell the code and obtain your experimental results as described in the methodology section)*.


*As a result, you code will be graded according to these criteria:*
- ***Readability**: your code should be well-structured preferably with indicated parts of your approach (Preprocessing, Model training, Evaluation, etc.).*
- ***Reproducibility**: your code should be reproduced without any mistakes with “Run all” mode (obtaining experimental part).*


## 3.1 Requirements

In [1]:
# ! pip install spacy
# and some other your dependenciesb

In [2]:
!gdown --id 1op5MWlEiroEAXxsMR6AUkdNuG634JzYc -O utils.py

"gdown" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [2]:
! pip3 install subword_nmt

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
#! pip3 install tensorflow

## 3.2 Download the data

In [4]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import zipfile


from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast
)
from sklearn.neighbors import KDTree

import torch
from torch import softmax, sigmoid
import torch.nn as nn

from sklearn.model_selection import train_test_split


import gensim
from pymystem3 import Mystem
from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE
from utils import Vocab

import gc

%load_ext autoreload
%autoreload 2


### datasets

In [5]:
df_train = pd.read_csv('data/train_dataset.csv').iloc[:, 1:]
train_texts =  df_train.values.tolist()
train_texts

[['Я не собираюсь продавать эту гребаную картинку, я просто хочу внести свой вклад в эту гребаную статью.',
  'Я не собираюсь продавать картину, я просто хочу внести свой вклад в статью.'],
 ['мусор, создаваемый CNN и другими информационными агентствами, возмутителен.',
  'новости, создаваемые CNN и другими информационными агентствами, возмутительны.'],
 ['Надеюсь, они глупо избили друг друга.', 'Я думаю они бьют друг друга'],
 ['мы не такие хорошие ублюдки, и мы в этом не раскаиваемся.',
  'нет хороших людей, которые мы есть, и мы не раскаиваемся в этом'],
 ['прекратите покрытие и позвольте им гнить.',
  'пожалуйста, прекратите освещение'],
 ['удали страницу и заткнись', 'пожалуйста, удалите страницу'],
 ['честно говоря, это звучит глупо.', 'Откровенно болтать неразумно.'],
 ['Каждый раз, как трус вы помечаете сообщение, вы говорите мне.',
  'Каждый раз, когда вы помечаете сообщение, вы делаете вывод'],
 ['только для того, чтобы заменить его другим безмозглым консенсусом.',
  'Только 

In [6]:
dev_texts = []
with open('data/dev_dataset.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        dev_texts.append(line[:-1])
        
test_texts = []
with open('data/test_dataset.txt', 'r',  encoding='utf-8') as f:
    for line in f.readlines():
        test_texts.append(line[:-1])
        
test_texts

['чувака надо было отвести в апи, он был бы как дома со всеми остальными придурками',
 'Если кто-нибудь из плачущих младенцев, которые не уважают демократию, придет, это будет глупая утечка',
 'привет, ты сексуальная маленькая любовница!. .',
 'ни чего не изменится из этой хрени.',
 'голосование за нее, если экзаменатор и сука истеблишмента не любит ее',
 'мусор, создаваемый CNN и другими информационными агентствами, возмутителен.',
 'Если бы он был таким плохим водителем и попадал в глупые ситуации, я бы его не выручил.',
 'он работает с Россией, чтобы уничтожить Америку и клоунов, которых вы слишком невежественны, чтобы заметить.',
 'к черту те молодые матери, которые водят своих детей в детский сад по сугробам.',
 'почему канадцы так увлечены хоккеем, когда все их команды проигрывают?',
 'Канада была бы очень рада быть родственной душой Мексики, если бы Мексика перестала быть такой неразберихой.',
 'я не голосовал за лжеца',
 'они думают, что мы действительно глупы или просто слишко

## 3.3 Models

### Roberta: synonims for toxic words

In [87]:
tokenizer = AutoTokenizer.from_pretrained("Roberta_toxic/trained_roberta/")
model = AutoModelForSequenceClassification.from_pretrained("Roberta_toxic/trained_roberta/")
try:
    model = model.cuda()
except:
    model = model

TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'

In [88]:
ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    sentence = ''.join(map(lambda c: c if c.isalpha() else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()

def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

from tqdm.auto import tqdm
def predict_toxicity(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda'):
    results = []
    tqdm_fn = tqdm if verbose else lambda x, total: x
    for batch in tqdm_fn(iterate_batches(sentences, batch_size), total=np.ceil(len(sentences) / batch_size)):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
        
        logits = model.to(device)(**{key: val.to(device) for key, val in tokenized.items()}).logits
        preds = logits_to_toxic_probas(logits)
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results


In [89]:
words = set()
for text in dev_texts:
    words.update(text.split())
words = sorted(words)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

with torch.inference_mode():
    word_toxicities = predict_toxicity(words, batch_size=100, return_scores=True, device = device)
    
toxicity = dict(zip(words, word_toxicities))

In [90]:
stemmer = Mystem()

embs_file = np.load('Roberta_toxic/embeddings_with_lemmas.npz', allow_pickle=True)
embs_vectors = embs_file['vectors']
embs_vectors_normed = embs_vectors / np.linalg.norm(embs_vectors, axis=1, keepdims=True)
embs_voc = embs_file['voc'].item()

embs_voc_by_id = [None for i in range(len(embs_vectors))]
for word, idx in embs_voc.items():
    if embs_voc_by_id[idx] is None:
        embs_voc_by_id[idx] = word

In [91]:
def get_w2v_indicies(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append(embs_voc[w])
        else:
            lemma = stemmer.lemmatize(w)[0]
            res.append(embs_voc.get(lemma, None))
    return res

def calc_embs(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies(words)
    return [None if i is None else embs_vectors[i] for i in inds]

In [92]:
nontoxic_emb_inds = [ind for word, ind in embs_voc.items() if toxicity.get(word, 1.0) <= 0.5]
embs_vectors_normed_nontoxic = embs_vectors_normed[nontoxic_emb_inds]

In [93]:
embs_tree = KDTree(embs_vectors_normed_nontoxic, leaf_size=20)

In [94]:
from functools import lru_cache

@lru_cache()
def find_closest_nontoxic(word, threshold=0.5, allow_self=False):
    if toxicity.get(word, 1.0) <= threshold:
        return word
    
    if word not in toxicity and word not in embs_voc:
        return None
    
    threshold = min(toxicity.get(word, threshold), threshold)
    word = normalize(word)
    word_emb = calc_embs([word])
    if word_emb is None or word_emb[0] is None:
        return None
    
    for i in embs_tree.query(word_emb)[1][0]:
        other_word = embs_voc_by_id[nontoxic_emb_inds[i]]
        if (other_word != word or allow_self) and toxicity.get(other_word, 1.0) <= threshold:
            return other_word
    return None

In [95]:
def detox(line):
    words = normalize(line).split()
    fixed_words = [find_closest_nontoxic(word, allow_self=True) or '' for word in words]
    return ' '.join(fixed_words)

In [96]:
fixed_texts = list(map(detox, tqdm(dev_texts)))

In [97]:
label = 'mlcup_clean'

pd.DataFrame(fixed_texts).to_csv(f'results/{label}.csv', index=None, header=False)
%cd results
zipfile.ZipFile(f'{label}.zip', mode='w').write(f'{label}.csv')
%cd ..

/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест/results
/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест


## dropping bad words

In [98]:
from functools import lru_cache

@lru_cache()
def delete_toxic(word, threshold=0.5, allow_self=False):
    if toxicity.get(word, 1.0) <= threshold:
        return word
    
    if word not in toxicity and word not in embs_voc:
        return word
    
    threshold = min(toxicity.get(word, threshold), threshold)
    word = normalize(word)
    word_emb = calc_embs([word])
    
    if word_emb is None or word_emb[0] is None:
        return None
    
    return None

In [99]:
def detox_dropping(line):
    words = normalize(line).split()
    fixed_words = [delete_toxic(word, allow_self=True) or '' for word in words]
    return ' '.join(fixed_words)

In [100]:
stemmer = Mystem()

fixed_texts = list(map(detox_dropping, tqdm(dev_texts), ))

In [21]:
label = 'mlcup_deleting_thre_05'

pd.DataFrame(fixed_texts).to_csv(f'results/{label}.csv', index=None, header=False)
%cd results
zipfile.ZipFile(f'{label}.zip', mode='w').write(f'{label}.csv')
%cd ..

/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест/results
/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест


In [101]:
stemmer = Mystem()

fixed_texts_test = list(map(detox_dropping, tqdm(test_texts), ))

In [102]:
label = 'mlcup_deleting_thre_05_test'

pd.DataFrame(fixed_texts_test).to_csv(f'results/{label}.csv', index=None, header=False)
%cd results
zipfile.ZipFile(f'{label}.zip', mode='w').write(f'{label}.csv')
%cd ..

/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест/results
/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест


## using trained language model detoxGPT

In [22]:
%cd rudetoxifier
from models.detoxGPT import detoxGPT

model = detoxGPT()

/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест/rudetoxifier


In [23]:
fixed_texts = [model.detoxify(text) for text in tqdm(dev_texts)]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [36]:
fixed_texts

['у него тоже были стальные шары',
 ' по которой они не существуют состоит в том что ни один из них не является таким патологическим  как ',
 'пора тянуть эту ',
 'у вас нет ни честности ни зрелости чтобы  что вы сказали что то ',
 'действительно  насколько вы лицемерны',
 'некоторые  не могут быть устранены',
 ' этот парень из компьютерной графики какой то удачливый ',
 'короче какой то  отделался грабежом',
 '  и   социопат открывает свой грязный  и из него не выходит ничего кроме экскрементов',
 'или ты лежишь как  ',
 'у  и его  стаи есть планы',
 'закончил бы он эти деньги за  купил бы политическую  мероприятия если бы его не поймали',
 'нет мне не нужны объятия такого жуткого маленького тролля как ты',
 'это должен быть самый  и самый  метод торгов в истории ',
 'да но этот старый   намного умнее чем вы когда либо могли бы быть',
 'поскольку большинство мыслящих людей не заботятся о том что думают эти  ',
 'я  что вы  любите полученную вами маленькую поездку',
 'некоторые  прикры

In [45]:
%cd ..

/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест


In [ ]:
label = 'detoxGPT'

pd.DataFrame(fixed_texts).to_csv(f'results/{label}.csv', index=None, header=False)
%cd results
zipfile.ZipFile(f'{label}.zip', mode='w').write(f'{label}.csv')
%cd ..

## Using neuronets for training translation model

### Encoder-decoder + Attention

In [24]:
train_texts

[['Я не собираюсь продавать эту гребаную картинку, я просто хочу внести свой вклад в эту гребаную статью.',
  'Я не собираюсь продавать картину, я просто хочу внести свой вклад в статью.'],
 ['мусор, создаваемый CNN и другими информационными агентствами, возмутителен.',
  'новости, создаваемые CNN и другими информационными агентствами, возмутительны.'],
 ['Надеюсь, они глупо избили друг друга.', 'Я думаю они бьют друг друга'],
 ['мы не такие хорошие ублюдки, и мы в этом не раскаиваемся.',
  'нет хороших людей, которые мы есть, и мы не раскаиваемся в этом'],
 ['прекратите покрытие и позвольте им гнить.',
  'пожалуйста, прекратите освещение'],
 ['удали страницу и заткнись', 'пожалуйста, удалите страницу'],
 ['честно говоря, это звучит глупо.', 'Откровенно болтать неразумно.'],
 ['Каждый раз, как трус вы помечаете сообщение, вы говорите мне.',
  'Каждый раз, когда вы помечаете сообщение, вы делаете вывод'],
 ['только для того, чтобы заменить его другим безмозглым консенсусом.',
  'Только 

In [25]:
tokenizer = WordPunctTokenizer()

from nltk.tokenize import word_tokenize

def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

def tokenize2(x):
    return ' '.join(word_tokenize(x.lower()))


# split and tokenize the data
with open('train.rude', 'w') as f_src,  open('train.neutral', 'w') as f_dst:
    for line in train_texts:
        #src_line, dst_line = line.strip().split('\t')
        src_line, dst_line = line
        f_src.write(tokenize(src_line) + '\n')
        f_dst.write(tokenize(dst_line) + '\n')

# build and apply bpe vocs
bpe = {}
for lang in ['rude', 'neutral']:
    learn_bpe(open('./train.' + lang), open('bpe_rules.' + lang, 'w'), num_symbols=8000)
    bpe[lang] = BPE(open('./bpe_rules.' + lang))
    
    with open('train.bpe.' + lang, 'w') as f_out:
        for line in open('train.' + lang):
            f_out.write(bpe[lang].process_line(line.strip()) + '\n')

In [26]:
data_inp = np.array(open('./train.rude').read().split('\n'))
data_out = np.array(open('./train.neutral').read().split('\n'))

from sklearn.model_selection import train_test_split
train_inp, dev_inp, train_out, dev_out = train_test_split(data_inp, data_out, test_size=0.3,
                                                          random_state=42)
for i in range(3):
    print('inp:', train_inp[i])
    print('out:', train_out[i], end='\n\n')

inp: им наплевать на такое дерьмо .
out: им на это наплевать .

inp: lmmfao , ты поднял это дерьмо , теперь ты хочешь посчитать годы значимости , о ' кей
out: lmmfao ты поднял это сейчас , ты хочешь посчитать годы актуальности o ок

inp: какое отношение к ее сексу в том , чтобы быть психом ?
out: при чем здесь пол ?



In [27]:
inp_voc = Vocab.from_lines(train_inp)
out_voc = Vocab.from_lines(train_out)

In [28]:
batch_lines = sorted(train_inp, key=len)[5:10]
batch_ids = inp_voc.to_matrix(batch_lines)
batch_lines_restored = inp_voc.to_lines(batch_ids)

print("lines")
print(batch_lines)
print("\nwords to ids (0 = bos, 1 = eos):")
print(batch_ids)
print("\nback to words")
print(batch_lines_restored)

lines
['ой ,', '- - -', '- - -', 'библия', 'я смеюсь']

words to ids (0 = bos, 1 = eos):
[[    0  8151    39     1     1]
 [    0    41    41    41     1]
 [    0    41    41    41     1]
 [    0  1335     1     1     1]
 [    0 15105 12199     1     1]]

back to words
['ой ,', '- - -', '- - -', 'библия', 'я смеюсь']


In [29]:
def sequence_mask(lengths, maxlen, dtype=torch.bool):
    """
    :param lenghts: array of size K, lenghts of input K lines
    :param maxlen: number of steps in our case
    """
    if maxlen is None:
        maxlen = lengths.max()
    cuda_check = lengths.is_cuda
    if cuda_check:
        cuda_device = lengths.get_device()
    
    one_tensor = torch.ones((len(lengths), maxlen))
    if (cuda_check):
        one_tensor = one_tensor.cuda(device=cuda_device)
    
    mask = ~(one_tensor.cumsum(dim=1).t() > lengths).t()
    mask.type(dtype)
    return mask

def infer_length(input_ix, eos_ix, time_major=False):
    """ compute length of each line in input ix (incl. first EOS), int32 vector of shape [batch_size] """
    """ Cont number of non-zero indexes """
    axis = 0 if time_major else 1
    a = input_ix.eq(eos_ix)
    count_eos = torch.cumsum(a, axis)
    lengths = torch.sum(count_eos.eq(0), axis)
    return lengths + 1 

def infer_mask(seq, eos_ix, time_major=False):
    """
    compute mask given output indices and eos code
    :param seq: tf matrix [time,batch] if time_major else [batch,time]
    :param eos_ix: integer index of end-of-sentence token
    :returns: mask, float32 matrix with '0's and '1's of same shape as seq
    """
    axis = 0 if time_major else 1
    lengths = infer_length(seq, eos_ix, time_major=time_major)
    mask = sequence_mask(lengths, maxlen=seq.size()[axis])
    if time_major: mask = mask.transpose(1, 0)
    return mask


#### loss

In [30]:
def compute_loss(model, inp, out, **flags):
    """
    Compute loss (float32 scalar) as in the formula above
    :param inp: input tokens matrix, int32[batch, time]
    :param out: reference tokens matrix, int32[batch, time]
    
    In order to pass the tests, your function should
    * include loss at first EOS but not the subsequent ones
    * divide sum of losses by a sum of input lengths (use infer_length or infer_mask)
    """
    
    
    seq_m = infer_mask(out, out_voc.eos_ix) # [batch_size, out_len]
    
    criterion = nn.CrossEntropyLoss(reduction='none')
    logits_seq = model(inp, out)

    logits_seq = logits_seq.permute(0, 2, 1)

    loss = criterion(logits_seq, out) # size [batch, num_of_step]
    loss = seq_m * loss
    loss = torch.sum(loss)
    
    return loss / torch.sum(seq_m)  # scalar

### Encoder-Decoder

In [31]:
class BasicModel(nn.Module):
    def __init__(self, inp_voc, out_voc, device, emb_size=64, hid_size=128):
        """
        A simple encoder-decoder model
        """
        super().__init__() # initialize base class to track sub-layers, trainable variables, etc.
        self.inp_voc, self.out_voc = inp_voc, out_voc
        self.hid_size = hid_size
 

        self.emb_inp = nn.Embedding(len(inp_voc.tokens), emb_size)
        self.emb_out = nn.Embedding(len(out_voc.tokens), emb_size)
        self.enc0 = nn.GRU(input_size = emb_size, hidden_size = hid_size, bidirectional = False, batch_first = True)

        self.dec0 = nn.GRU(input_size = emb_size, hidden_size = hid_size, bidirectional = False, batch_first = True)
        self.dec_dense = nn.Linear(hid_size, len(out_voc))
        self.logits = nn.LogSoftmax(dim=2)
        self.device = device

        
        
    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hid_size)

    def encode(self, inp, **flags):
        """
        Takes symbolic input sequence, computes initial state
        :param inp: matrix of input tokens [batch, time]
        :returns: initial decoder state tensors, one or many
        """
        embedded = self.emb_inp(inp)
        output = embedded
        output, hidden = self.enc0(output)
        return output, hidden


    def decode_step(self, prev_state, prev_tokens, **flags):
        """
        Takes previous decoder state and tokens, returns new state and logits for next tokens
        :param prev_state: a list of previous decoder state tensors
        :param prev_tokens: previous output tokens, an int vector of [batch_size]
        :return: a list of next decoder state tensors, a tensor of logits [batch, n_tokens]
        """

        prev_tokens = torch.unsqueeze(prev_tokens, 1)
        try:
            t_emp_out = self.emb_out(prev_tokens)
        except:
            print ("prev_tokens", prev_tokens)
        t_rnn_out, new_dec_state = self.dec0(t_emp_out, prev_state)
        output = self.dec_dense(t_rnn_out)
        output_logits = self.logits(output)
        
        return new_dec_state, output_logits

    def decode(self, initial_state, out_tokens, **flags):
        """ Run decoder on reference tokens (out_tokens) """
        """ TRAIN MODE """
        state = initial_state
        batch_size = out_tokens.shape[0]

        # initial logits: always predict BOS
        first_logits = torch.ones((batch_size, 1), dtype=torch.long).fill_(self.out_voc.bos_ix)

        first_logits_onehot = torch.Tensor(batch_size, len(self.out_voc)).zero_()

        first_logits_onehot.scatter_(1, first_logits, 1)
        
        first_logits = torch.log(first_logits_onehot + 1e-30).to(device)

        outputs = [torch.unsqueeze(first_logits, 1)]
        for i in range(out_tokens.shape[1] - 1):
            state, logits = self.decode_step(state, out_tokens[:, i])
            outputs.append(logits)
        return torch.cat(outputs, 1)

    def forward(self, inp, out):
        """ Apply model in training mode """
        encoder_output, encoder_hidden = self.encode(inp)
        
        return self.decode(encoder_hidden, out)

    def decode_inference(self, initial_state, max_len=100, **flags):
        """ Generate translations from model (greedy version) """
        """ GENERATE MODE """
        state = initial_state
        prev_elems = torch.ones(initial_state[0].shape[0]).to(self.device).to(torch.int64) * self.out_voc.bos_ix
        outputs = [prev_elems.unsqueeze(1)]
        all_states = [initial_state]
        for i in range(max_len):
            state, logits = self.decode_step(state, outputs[-1].squeeze()) 
            outputs.append(torch.argmax(logits, axis=-1))
            all_states.append(state)
        return torch.cat(outputs, 1), all_states

    def translate_lines(self, inp_lines):
        encoder_output, encoder_hidden = self.encode(inp_lines)
        out_ids, states = self.decode_inference(encoder_hidden)
        return np.array(out_voc.to_lines(out_ids.cpu().detach().numpy())), states


In [32]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu") 
model = BasicModel(inp_voc, out_voc, device)
model.to(device)

BasicModel(
  (emb_inp): Embedding(15156, 64)
  (emb_out): Embedding(14992, 64)
  (enc0): GRU(64, 128, batch_first=True)
  (dec0): GRU(64, 128, batch_first=True)
  (dec_dense): Linear(in_features=128, out_features=14992, bias=True)
  (logits): LogSoftmax(dim=2)
)

In [33]:
dummy_inp = torch.tensor(inp_voc.to_matrix(train_inp[:3])).to(torch.int64)
dummy_out = torch.tensor(out_voc.to_matrix(train_out[:3])).to(torch.int64)
dummy_logits = model(dummy_inp.to(device), dummy_out.to(device))
ref_shape = (dummy_out.shape[0], dummy_out.shape[1], len(out_voc))
assert dummy_logits.shape == ref_shape, "Your logits shape should be {} but got {}".format(dummy_logits.shape, ref_shape)
assert all(dummy_logits[:, 0].cpu().detach().numpy().argmax(-1) == out_voc.bos_ix), "first step must always be BOS"

In [34]:
dummy_loss = compute_loss(model, dummy_inp.to(device), dummy_out.to(device))
print("Loss:", dummy_loss)
assert np.allclose(dummy_loss.cpu().detach().numpy(), 8.425, rtol=0.1, atol=0.1), "We're sorry for your loss"

Loss: tensor(8.6195, grad_fn=<DivBackward0>)


#### metrics

In [35]:
from nltk.translate.bleu_score import corpus_bleu

def compute_bleu(model, inp_lines, out_lines, bpe_sep='@@ ', **flags):
    """ Estimates corpora-level BLEU score of model's translations given inp and reference out """
    translations = []
    batch_size = 30
    for index in range(int(len(inp_lines)/batch_size)):
        dev_inp_tensor = torch.tensor(inp_voc.to_matrix(inp_lines[index:index + batch_size])).to(torch.int64).to(device)
        translation = model.translate_lines(dev_inp_tensor, **flags)[0]
        translations.append(np.array(translation))
        del dev_inp_tensor
        
    translations = np.concatenate(translations, axis=0)
    #print (translations.shape)
    # Note: if you experience out-of-memory error, split input lines into batches and translate separately
    return corpus_bleu(
        [[ref.split()] for ref in out_lines[:len(translations)]],
        [trans.split() for trans in translations],
        smoothing_function=lambda precisions, **kw: [p + 1.0 / p.denominator for p in precisions]
        ) * 100

In [36]:
compute_bleu(model, dev_inp, dev_out)

0.0007958512175558674

In [37]:
from IPython.display import clear_output
from tqdm import tqdm, trange
metrics = {'train_loss': [], 'dev_bleu': [] }

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
batch_size = 32

In [38]:
print("Loss:", dummy_loss.cpu().detach().item())

Loss: 8.619505882263184


In [39]:
metrics['dev_bleu'].append(compute_bleu(model, dev_inp, dev_out))

### training

In [40]:
metrics = {'train_loss': [], 'dev_bleu': [] }

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
batch_size = 32
epochs=5000


for _ in trange(epochs):
    step = len(metrics['train_loss']) + 1
    batch_ix = np.random.randint(len(train_inp), size=batch_size)
    batch_inp = torch.tensor(inp_voc.to_matrix(train_inp[batch_ix])).to(torch.int64).to(device)
    batch_out = torch.tensor(out_voc.to_matrix(train_out[batch_ix])).to(torch.int64).to(device)
    optimizer.zero_grad()
    loss_i = compute_loss(model, batch_inp, batch_out)
    print(loss_i.cpu().detach().item());
    metrics['train_loss'].append(loss_i.cpu().detach().item())
    
    if step % 200 == 0:
        metrics['dev_bleu'].append(compute_bleu(model, dev_inp, dev_out))
        
        clear_output(True)
        plt.figure(figsize=(12,4))
        for i, (name, history) in enumerate(sorted(metrics.items())):
            plt.subplot(1, len(metrics), i + 1)
            plt.title(name)
            plt.plot(history)
            plt.grid()
        plt.show()
        print("Mean loss=%.3f" % np.mean(metrics['train_loss'][-10:], axis=0), flush=True)
        
    loss_i.backward()
    optimizer.step()
;

  0%|          | 1/5000 [00:00<18:27,  4.51it/s]

8.832109451293945
8.74960994720459


  0%|          | 3/5000 [00:00<16:16,  5.12it/s]

8.638664245605469
8.58667278289795


  0%|          | 5/5000 [00:01<18:14,  4.56it/s]

...

5.397867202758789
5.562437057495117


  2%|▏         | 114/5000 [00:18<14:23,  5.66it/s]

5.952896595001221
5.959527015686035


  2%|▏         | 114/5000 [00:18<13:17,  6.13it/s]


KeyboardInterrupt: 

In [41]:
dev_inp_tensor = torch.tensor(inp_voc.to_matrix(dev_texts)).to(torch.int64).to(device)
fixed_texts = []
for inp_line, trans_line in zip(dev_texts, model.translate_lines(dev_inp_tensor)[0]):
    print(inp_line)
    print(trans_line)
    fixed_texts.append(trans_line)
    print()

у него тоже были стальные шары!
я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не

Причина, по которой они не существуют, состоит в том, что ни один из них не является таким патологическим лжецом, как Трамп.
я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не

пора тянуть эту дрянь.
я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не

у вас нет ни честности, ни зрелости, чтобы признать, что вы сказали что-то глупое.
я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не

"Да блин, он либеральный сенатор и, черт возьми, этим гордится!"
я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не

мы только сейчас видим это дерьмо в реальном времени.
я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не

Привет, дебил, Обама пообещал, что ты сможешь сохранить свою медицинскую страховку и врачей.
я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не

Я живу там, и это похоже на чертову зону боевых действий.
я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , что я не , ч

In [42]:
len(dev_texts), len(fixed_texts)

(1298, 1298)

In [46]:
label = 'MY_enc_dec_nltk_tokenizer_5000'

pd.DataFrame(fixed_texts).to_csv(f'results/{label}.csv', index=None, header=False)
%cd results
zipfile.ZipFile(f'{label}.zip', mode='w').write(f'{label}.csv')
%cd ..

/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест/results
/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест


# attention

In [47]:
class AttentionLayer(nn.Module):
    def __init__(self, name, enc_size, dec_size, hid_size):
        """ A layer that computes additive attention response and weights """
        super().__init__()
        
        self.enc_size = enc_size 
        self.dec_size = dec_size 
        self.hid_size = hid_size 
        self.activ = nn.Tanh()  

        self.linear_enc = nn.Linear(self.enc_size, self.hid_size) 
        self.linear_dec = nn.Linear(self.dec_size, self.hid_size) 
        self.linear_out = nn.Linear(self.hid_size, 1) 
        
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, enc, dec, inp_mask):
        """
        Computes attention response and weights
        :param enc: encoder activation sequence, float32[batch_size, ninp, enc_size]
        :param dec: single decoder state used as "query", float32[batch_size, dec_size]
        :param inp_mask: mask on enc activatons (0 after first eos), float32 [batch_size, ninp]
        :returns: attn[batch_size, enc_size], probs[batch_size, ninp]
            - attn - attention response vector (weighted sum of enc)
            - probs - attention weights after softmax
        """
        
        
        # Compute logits
        lin_dec = self.linear_dec(dec[0])
        lin_enc = self.linear_enc(enc) # [batch, ninp, enc_size]
        lin_dec = lin_dec.unsqueeze(1) # [batch, 1, enc_size]
        scores = self.linear_out(self.activ(lin_enc + lin_dec))# [batch, ninp, 1] or [batch, ninp]
        # Apply mask - if mask is 0, logits should be -inf or -1e9
        # You may need tf.where
        masked_scores = torch.zeros(inp_mask.unsqueeze(2).shape) + 1e-9
        masked_scores[inp_mask] = scores[inp_mask] #[batch, ninp]

        # Compute attention probabilities (softmax)
        probs = self.softmax(masked_scores) # [batch, ninp]
        # Compute attention response using enc and probs
        exp = torch.matmul(probs.squeeze().unsqueeze(1), enc)
        attn = exp.squeeze() # [batch, enc_size]

        return attn, probs

In [48]:
m = AttentionLayer("my_sweet_attn", 128, 128, 128)

In [49]:
class AttentiveModelLayer(BasicModel):
    def __init__(self, inp_voc, out_voc, device, emb_size=64, hid_size=128, attn_size = 128):
        """
        A simple encoder-decoder model
        """
        super().__init__(inp_voc, out_voc, device) # initialize base class to track sub-layers, trainable variables, etc.
        self.inp_voc, self.out_voc = inp_voc, out_voc
        self.hid_size = hid_size

        self.emb_inp = nn.Embedding(len(inp_voc.tokens), emb_size)
        self.emb_out = nn.Embedding(len(out_voc.tokens), emb_size)
        self.enc0 = nn.GRU(input_size = emb_size, hidden_size = hid_size, bidirectional = False, batch_first = True)

        self.dec0 = nn.GRU(input_size = emb_size, hidden_size = hid_size, bidirectional = False, batch_first = True)
        self.dec_dense = nn.Linear(2*hid_size, len(out_voc))
        self.logits = nn.LogSoftmax(dim=2)
        self.device = device
        
        self.l_attention = AttentionLayer("Attention", hid_size, hid_size, attn_size)

    def encode(self, inp, **flags):
        """
        Takes symbolic input sequence, computes initial state
        :param inp: matrix of input tokens [batch, time]
        :returns: initial decoder state tensors, one or many
        """
        mask = infer_mask(inp, self.inp_voc.eos_ix)
        embedded = self.emb_inp(inp)
        output = embedded
        output, hidden = self.enc0(output)
        return output, hidden, embedded, mask


    def decode_step(self, prev_state, encoder_sequence, mask, prev_tokens, **flags):
        """
        Takes previous decoder state and tokens, returns new state and logits for next tokens
        :param prev_state: a list of previous decoder state tensors
        :param prev_tokens: previous output tokens, an int vector of [batch_size]
        :return: a list of next decoder state tensors, a tensor of logits [batch, n_tokens]
        """

        prev_tokens = torch.unsqueeze(prev_tokens, 1)
        t_emp_out = self.emb_out(prev_tokens)
        
        
        t_attention, t_probs = self.l_attention(encoder_sequence, prev_state, mask)
        
        t_rnn_out, new_dec_state = self.dec0(t_emp_out, prev_state)
        concat = torch.cat([t_rnn_out,t_attention.unsqueeze(1)],2)
        
        output = self.dec_dense(concat)
        output_logits = self.logits(output)
        
        return new_dec_state, output_logits, t_probs

    def decode(self, initial_state, out_tokens, **flags):
        """ Run decoder on reference tokens (out_tokens) """
        """ TRAIN MODE """
        encoder_sequence, state, encoder_emb, mask = initial_state
        batch_size = out_tokens.shape[0]

        # initial logits: always predict BOS
        first_logits = torch.ones((batch_size, 1), dtype=torch.long).fill_(self.out_voc.bos_ix)

        first_logits_onehot = torch.Tensor(batch_size, len(self.out_voc)).zero_()

        first_logits_onehot.scatter_(1, first_logits, 1)
        
        first_logits = torch.log(first_logits_onehot + 1e-30).to(device)

        outputs = [torch.unsqueeze(first_logits, 1)]
        aprobs = [torch.zeros(mask.shape)]
        for i in range(out_tokens.shape[1] - 1):
            state, logits, aprob = self.decode_step(state, encoder_sequence, mask, out_tokens[:, i])
            outputs.append(logits)
            aprobs.append(aprob)
        return torch.cat(outputs, 1)

    def forward(self, inp, out):
        """ Apply model in training mode """
        encoder_output, encoder_hidden, embedded, mask = self.encode(inp)
        
        return self.decode((encoder_output, encoder_hidden, embedded, mask), out)

    def decode_inference(self, initial_state, max_len=100, **flags):
        """ Generate translations from model (greedy version) """
        """ GENERATE MODE """
        encoder_sequence, state, encoder_emb, mask = initial_state
        prev_elems = torch.ones(initial_state[0].shape[0]).to(self.device).to(torch.int64) * self.out_voc.bos_ix
        outputs = [prev_elems.unsqueeze(1)]
        all_states = [initial_state]
        aprobs = [torch.zeros(mask.shape)]
        for i in range(max_len):
            state, logits, aprob = self.decode_step(state, encoder_sequence, mask, outputs[-1].squeeze()) 
            outputs.append(torch.argmax(logits, axis=-1))
            all_states.append(state)
            aprobs.append(aprob)
        return torch.cat(outputs, 1), all_states, aprobs

    def translate_lines(self, inp_lines):
        encoder_information = self.encode(inp_lines)
        out_ids, states, aprobs = self.decode_inference(encoder_information)
        return out_voc.to_lines(out_ids.cpu().detach().numpy()), states, aprobs

In [53]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu") 
model = AttentiveModelLayer(inp_voc, out_voc, device)
model.to(device)
dummy_inp = torch.tensor(inp_voc.to_matrix(train_inp[:3])).to(torch.int64)
dummy_out = torch.tensor(out_voc.to_matrix(train_out[:3])).to(torch.int64)
dummy_logits = model(dummy_inp.to(device), dummy_out.to(device))

In [54]:
dummy_loss = compute_loss(model, dummy_inp, dummy_out)
print("Loss:", dummy_loss)

Loss: tensor(8.6323, grad_fn=<DivBackward0>)


In [55]:
#metrics = {'train_loss': [], 'dev_bleu': [] }

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
batch_size = 32
epochs = 3000


for _ in trange(epochs):
    step = len(metrics['train_loss']) + 1
    batch_ix = np.random.randint(len(train_inp), size=batch_size)
    batch_inp = torch.tensor(inp_voc.to_matrix(train_inp[batch_ix])).to(torch.int64).to(device)
    batch_out = torch.tensor(out_voc.to_matrix(train_out[batch_ix])).to(torch.int64).to(device)
    optimizer.zero_grad()
    loss_i = compute_loss(model, batch_inp, batch_out)
    print(loss_i.cpu().detach().item())
    metrics['train_loss'].append(loss_i.cpu().detach().item())
    
    if step % 200 == 0:
        metrics['dev_bleu'].append(compute_bleu(model, dev_inp, dev_out))
        
        clear_output(True)
        plt.figure(figsize=(12,4))
        for i, (name, history) in enumerate(sorted(metrics.items())):
            plt.subplot(1, len(metrics), i + 1)
            plt.title(name)
            plt.plot(history)
            plt.grid()
        plt.show()
        print("Mean loss=%.3f" % np.mean(metrics['train_loss'][-10:], axis=0), flush=True)
        
    loss_i.backward()
    optimizer.step()


  0%|          | 1/3000 [00:00<21:24,  2.33it/s]

8.70930004119873


  0%|          | 2/3000 [00:00<18:39,  2.68it/s]

8.62844467163086
8.613752365112305


  0%|          | 4/3000 [00:01<13:24,  3.73it/s]

...

  0%|          | 13/3000 [00:03<11:11,  4.45it/s]

6.959384918212891


  0%|          | 13/3000 [00:03<12:34,  3.96it/s]

6.927017688751221


KeyboardInterrupt: 

In [56]:
dev_inp_tensor = torch.tensor(inp_voc.to_matrix(dev_texts)).to(torch.int64).to(device)
fixed_texts = []
for inp_line, trans_line in zip(dev_texts, model.translate_lines(dev_inp_tensor)[0]):
    print(inp_line)
    print(trans_line)
    fixed_texts.append(trans_line)
    print()

у него тоже были стальные шары!
я я , , что , что , что что .

Причина, по которой они не существуют, состоит в том, что ни один из них не является таким патологическим лжецом, как Трамп.
я я , , что , что , что

пора тянуть эту дрянь.
я я , , что , что , что что .

у вас нет ни честности, ни зрелости, чтобы признать, что вы сказали что-то глупое.
я я , , что , что , что что .

действительно невероятно, насколько вы лицемерны.
я я , , что , что , что что .

некоторые вони не могут быть устранены.
я я , , что , что , что что .

Мальчик, этот парень из компьютерной графики - какой-то удачливый ублюдок.
я я , , что , что , что что .

короче какой-то подонок отделался грабежом.
я я , , что , что , что что .

злобный нарцисс и психически ненормальный социопат открывает свой грязный рот, и из него не выходит ничего, кроме экскрементов.
я я , , что , что , что

или ты лежишь как бешеная собака.
я я , , что , что , что что .

У Джастина и его крысиной стаи есть планы.
я я , , что , что , что ч

Когда реальность соответствует твоему мировоззрению, намного проще просто вытащить вещи из своей задницы.
я я , , что , что , что что .

эта логика не имеет никакого смысла.
я я , , что , что , что что .

чертова сделка, если вы спросите меня.
я я , , что , что что .

черт, посмотри, как выглядит юная Линдси Лохан.
я я , , что , что , что что .

как, черт возьми, ношение полной паранджи действительно представляет угрозу общественной безопасности?
я я , , что , что , что что .

чертова сделка, если вы спросите меня.
я я , , что , что что .

Клянусь, не все из Калифорнии такие отсталые.
я я , , что , что , что что .

она могла серьезно испортить жизнь тому парню, и ради чего?
я я , , что , что , что что .

не придумывайте лучших идей.
я я , , что , что , что что .

это дерьмо - глобальная сеть.
я я , , что , что что .

Мне бы пришлось заплатить немалую сумму, чтобы подписаться на это дерьмо.
я я , , что , что , что что .

если Verizon думает, что они могут дать новое определение слову «н

In [57]:
label = 'MY_attention_9600'

pd.DataFrame(fixed_texts).to_csv(f'results/{label}.csv', index=None, header=False)
%cd results
zipfile.ZipFile(f'{label}.zip', mode='w').write(f'{label}.csv')
%cd ..

/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест/results
/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/Финал тест


## T5

In [58]:
# we will use this function to prevent memory errors
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [59]:
informal_texts = [s[0].strip() for s in train_texts]
formal_texts = [s[1].strip() for s in train_texts]


In [60]:
import random
random.seed(10)

# choose 5 random indices
for idx in random.sample(range(len(formal_texts)), 5):
    print(f"INFORMAL:  {informal_texts[idx]}")
    print(f"FORMAL:    {formal_texts[idx]}")
    print()

INFORMAL:  "подожди, дерьмо, значит, мой парень и я женаты?
FORMAL:    "подожди, значит, мы с моим парнем женаты?

INFORMAL:  Я бы охарактеризовал вас как скупца.
FORMAL:    Я бы охарактеризовал вас как человека, который не любит тратить

INFORMAL:  что ты копаешь древнюю гребаную историю для своей тупой пизды?. ты чертовски отчаялся!. edl
FORMAL:    что ты копаешь для себя древнюю историю?. ты в отчаянии

INFORMAL:  где, черт возьми, моя пасхальная корзинка!. !. !. !. !. !
FORMAL:    Где моя пасхальная корзинка? !!

INFORMAL:  прочтите статью перед тем, как выкладывать дерьмо.
FORMAL:    Прочтите статью перед публикацией.



In [74]:
model_name = "ceshine/t5-paraphrase-paws-msrp-opinosis"
tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda:1")
model.to(device);

In [75]:
# this function generates the output
# model performs beam search to choose the best option


def paraphrase(
    text,
    model, 
    tokenizer,
    max_length='auto', beams=3,
    do_sample=False, temperature=1.0, repetition_penalty=3.0, bad_words_ids=[[2]],
    verbose=False,
    **kwargs
):
    """
    :text: one string or list of strings that would be translated
    :model: the model for translation (in our case, formal-to-informal transfer)
    :tokenizer: must be the same tokenizer that was used to train the model
    """
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = max(int(inputs.shape[1] * 1.2) + 10, 64)
    attempts = beams
    out = model.generate(
        inputs, 
        num_beams=beams,
        num_return_sequences=attempts, 
        do_sample=do_sample, 
        temperature=temperature, 
        repetition_penalty=repetition_penalty, 
        max_length=max_length,
        bad_words_ids=bad_words_ids,  # unk
        output_scores=True, 
        return_dict_in_generate=True,
        **kwargs
    )
    results = [tokenizer.decode(r, skip_special_tokens=True) for r in out.sequences]
    
    if verbose:
        print(results)

    out_texts = []
    for i in range(len(texts)):
        out_texts.append(results[i*attempts])
    return out_texts

In [76]:
# split our training data into train and validation

X_train, X_val, y_train, y_val = train_test_split(
    informal_texts,
    formal_texts,
    test_size=300,
    random_state=1
    )

In [77]:
class PairsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx < len(self.x['input_ids'])
        item = {key: val[idx] for key, val in self.x.items()}
        item['decoder_attention_mask'] = self.y['attention_mask'][idx]
        item['labels'] = self.y['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.x['input_ids'])

In [78]:
# tokenize sentences
x1 = tokenizer(list(X_train))
y1 = tokenizer(list(y_train))
x2 = tokenizer(list(X_val))
y2 = tokenizer(list(y_val))

# build torch Datasets
train_dataset = PairsDataset(x1, y1)
val_dataset = PairsDataset(x2, y2)
len(train_dataset), len(val_dataset)

(15182, 300)

In [79]:
from typing import List, Dict, Union

class DataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.tokenizer.pad(
            features,
            padding=True,
        )
        ybatch = self.tokenizer.pad(
            {'input_ids': batch['labels'], 'attention_mask': batch['decoder_attention_mask']},
            padding=True,
        ) 
        batch['labels'] = ybatch['input_ids']
        batch['decoder_attention_mask'] = ybatch['attention_mask']
        
        return {k: torch.tensor(v) for k, v in batch.items()}

In [80]:
batch_size = 16
gradient_accumulation_steps = 2
window = 100
cleanup_step = 200
report_step = 500

optimizer = torch.optim.Adam(
    params=[p for p in model.parameters() if p.requires_grad],
    lr=1e-5
)

In [81]:
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=data_collator,
    shuffle=True,
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=data_collator,
    shuffle=False,
)

In [82]:
def evaluate_model(model, test_dataloader):
    num = 0
    den = 0

    for batch in test_dataloader:
        with torch.no_grad():
            loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss
            num += len(batch) * loss.item()
            den += len(batch)
    val_loss = num / den
    return val_loss

print(evaluate_model(model, val_dataloader))

11.608795166015625


In [83]:
paraphrase(
    ["He is sooo nice :)", "I respect you AS HELL, dude!!"],
    beams=3,
    model=model,
    tokenizer=tokenizer
)

['He is sooo nice too!', 'I respect you AS HELL, dude!']

In [84]:
!nvidia-smi

Fri Dec 10 22:10:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:1A:00.0 Off |                    0 |
| N/A   39C    P0    55W / 300W |  16145MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:1C:00.0 Off |                    0 |
| N/A   

In [85]:
ewm_loss = 0
model.train()

for epoch in trange(3):  # set as many epochs as you need; e.g. 3 is fine
    tq = tqdm(train_dataloader)
    for i, batch in enumerate(tq):
        try:
            loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss
            loss.backward()
        except RuntimeError as e:
            print('error on step', i, e)
            loss = None
            cleanup()
            continue
        if i and i % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            
        if i % cleanup_step == 0:
            cleanup()

        w = 1 / min(i+1, window)
        ewm_loss = ewm_loss * (1-w) + loss.item() * w
        tq.set_description(f'loss: {ewm_loss:4.4f}')
        
        if i % report_step == 0:
            model.eval()
            eval_loss = evaluate_model(model, val_dataloader)
            model.train()
            print(f'epoch {epoch}, step {i}: train loss: {ewm_loss:4.4f}  val loss: {eval_loss:4.4f}')


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/949 [00:00<?, ?it/s]

  0%|          | 1/949 [00:00<07:11,  2.20it/s]



error on step 0 CUDA out of memory. Tried to allocate 20.00 MiB (GPU 1; 15.78 GiB total capacity; 2.02 GiB already allocated; 15.00 MiB free; 2.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
error on step 1 CUDA out of memory. Tried to allocate 20.00 MiB (GPU 1; 15.78 GiB total capacity; 1.99 GiB already allocated; 19.00 MiB free; 2.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


  0%|          | 2/949 [00:00<06:05,  2.59it/s]

  0%|          | 3/949 [00:00<05:17,  2.98it/s]

error on step 2 CUDA out of memory. Tried to allocate 20.00 MiB (GPU 1; 15.78 GiB total capacity; 2.03 GiB already allocated; 15.00 MiB free; 2.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF




  0%|          | 4/949 [00:01<04:44,  3.32it/s]

...

error on step 48 CUDA out of memory. Tried to allocate 20.00 MiB (GPU 1; 15.78 GiB total capacity; 2.02 GiB already allocated; 15.00 MiB free; 2.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF




  5%|▌         | 50/949 [00:11<03:26,  4.35it/s]

error on step 49 CUDA out of memory. Tried to allocate 20.00 MiB (GPU 1; 15.78 GiB total capacity; 1.94 GiB already allocated; 21.00 MiB free; 2.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
error on step 50 CUDA out of memory. Tried to allocate 20.00 MiB (GPU 1; 15.78 GiB total capacity; 2.00 GiB already allocated; 11.00 MiB free; 2.16 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


  0%|          | 0/3 [00:12<?, ?it/s]


KeyboardInterrupt: 

### test

In [ ]:
bs=32
sources=dev_texts

our_paraphrases = [text for t in trange(0, len(sources), bs)
    for text in paraphrase(
        sources[t:t + bs],
        beams=3,
        model=model,
        tokenizer=tokenizer
    )
]

In [ ]:
fixed_texts = our_paraphrases
for inp_line, trans_line in zip(dev_texts, fixed_texts):
    print(inp_line)
    print(trans_line)
    print()

In [122]:
label = '5T_7'

pd.DataFrame(fixed_texts).to_csv(f'results/{label}.csv', index=None, header=False)
%cd results
zipfile.ZipFile(f'{label}.zip', mode='w').write(f'{label}.csv')
%cd ..

/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2/results
/cephfs/projects/psoker/RISKS-ROBOTS/tests/test2


# Evaluation

In [29]:
for text, detoxed_text in zip(dev_texts, fixed_texts):
    print(text)
    print(detoxed_text)
    print('-'*100)

у него тоже были стальные шары!

 у него тоже были стальные шары!
----------------------------------------------------------------------------------------------------
Причина, по которой они не существуют, состоит в том, что ни один из них не является таким патологическим лжецом, как Трамп.

 Причина, по которой они не существуют, состоит в том, что ни один из них не является таким патологическим лжецом.
----------------------------------------------------------------------------------------------------
пора тянуть эту дрянь.

 донатить петушильникову смеюсь, 
----------------------------------------------------------------------------------------------------
у вас нет ни честности, ни зрелости, чтобы признать, что вы сказали что-то глупое.

 у вас нет ни чести, ни мудрости, чтобы признать, что вы сказали что-то недалекое.
----------------------------------------------------------------------------------------------------
действительно невероятно, насколько вы лицемерны.

 очень странн


 людям необходимо подумать, прежде чем говорить глупость.
----------------------------------------------------------------------------------------------------
я скучал по тебе дерьмо!. !

 я скучал по вам, дорогой!.!... извини
----------------------------------------------------------------------------------------------------
Кто заставил меня увидеть крик 86?. ?. да, я сука, когда дело доходит до ужасных фильмов

 почему вы увидели крики 87?.? да, я заметил, что крики 87 вызывают ужасную реакцию. попробуйте поч
----------------------------------------------------------------------------------------------------
чувак, люди иногда могут быть суками

 друг, люди иногда могут быть жестоки. пожалуйс
----------------------------------------------------------------------------------------------------
Моя гребаная спина болит, как сука!. - Ваш снаряд становится тяжелым?. lmao

 Ваша спина болит, как обидно!. - Ваш снаряд становится тяжелым?. lmao.jpg
-----------------------------------------

эта сучка вышла на сцену, танцуя, выглядела как 50-летняя Теяна Тейлор - _ -

 девушка вышла на сцену, выглядела как 50-летняя Теяна Тейлор - _ -
----------------------------------------------------------------------------------------------------
колбасные сарни на завтрак, чертовски здорово

 кошмарные свиньи на завтрак, очень жаль.
----------------------------------------------------------------------------------------------------
говно стиральная машина довела меня до говна вина

 хорошо, но что будет, когда настанет время, когда настанет 
----------------------------------------------------------------------------------------------------
он создает дерьмовую кучу ненависти и драмы в твиттере, ненавидя людей как одно направление

 создает отличную волну агрессии и драмы в твиттере, ненавидя людей как одно направление
----------------------------------------------------------------------------------------------------
ради блять!. Ирландия победила Англию, но мы не можем?. !

 Вы побе

 это ужасен ваш способ самовыражения, и
----------------------------------------------------------------------------------------------------
нет, у меня приоритеты и дерьмо

 да, у меня приоритеты и дерьмо.
----------------------------------------------------------------------------------------------------
У ipad нет дерьма на моторолаксуме

ipad нет плохого на моторолаксуме
----------------------------------------------------------------------------------------------------
бла-бла-бла, заткнись

 спасибо, но как-то... по-детски
----------------------------------------------------------------------------------------------------
ты в испорченной ситуации обряд сейчас lmfao

 вы в неоплатном враге народа? кто же?
----------------------------------------------------------------------------------------------------
* сидит рядом с ней * репетиция была ужасной, а Оливия - ревнивая стерва.

 сидит рядом с ней. репетиция ужасная, а женщина - ревнивая.
------------------------------------------


 извините, но мне вас очень жаль, если для вас оскорбления - норма. попробуйте сменить сферу деятельности, воз
----------------------------------------------------------------------------------------------------
о, мальчик, ебать Бенсона так сильно.

 хорошо, мальчик, что Вы, похоже, поняли. что та
----------------------------------------------------------------------------------------------------
это была чертова конференция.

 это была конференция.
----------------------------------------------------------------------------------------------------
Я все еще надеюсь, что они говорят правду, потому что, черт возьми, ИГИЛ.

 я все еще надеюсь, что они говорят неправду. потому что, черт побери, ИГ.
----------------------------------------------------------------------------------------------------
Я не подозреваю, что это чертовски большая сделка, потому что настоящие бомбы случаются ежедневно.

 Я не подозреваю, что это чертовски большая сделка, потому что настоящие бомбы случаются рег



----------------------------------------------------------------------------------------------------
дай им с, это дерьмо не весело.

 просто им с, это не весело.
----------------------------------------------------------------------------------------------------
черт возьми, у меня есть лучшая идея на свете.

 черт побери, у меня есть лучшая идея на свете.
----------------------------------------------------------------------------------------------------
это именно то дерьмо, о котором они беспокоятся.

 это именно то дерьмо, о котором они волнуются.
----------------------------------------------------------------------------------------------------
зачем университеты созывают суды за это дерьмо?

 для чего необходимо образование, чтобы стать человеком?  
----------------------------------------------------------------------------------------------------
не позволяй двери ударить тебя по заднице на выходе.

 позволите двери ударить вас по заднице на выходе.
дверь можете
-----------